In [21]:
from transforna import GeneEmbeddModel,load,Results_Handler,RnaTokenizer
import yaml
import torch
import pickle
import numpy as np
mapping_dict_path: str = '/media/ftp_share/hbdx/data_for_upload/TransfoRNA/data/subclass_to_annotation.json'
model = "Seq-Rev"
model_name = f"Yak-hbdx/{model}-TransfoRNA"
model_dir = f"/nfs/home/yat_ldap/VS_Projects/TransfoRNA-Framework/models/tcga/TransfoRNA_FULL/sub_class/{model}/"
model_path = model_dir+"/ckpt/model_params_tcga.pt"
model_config_path = model_dir+"meta/hp_settings.yaml"
knn_path = model_dir+"/post_models/knn_model.sav"
cfg = load(model_config_path)
mapping_dict = load(mapping_dict_path)
#results_handler = Results_Handler(embedds_path = model_dir+"embedds",splits=['train'])
#results_handler.get_knn_model()
with open(model_dir+"/seq_tokens_ids_dict.yaml") as file:
    token_to_ids = yaml.load(file, Loader=yaml.FullLoader)
if 'struct' in model.lower():
    with open(model_dir+"/second_input_tokens_ids_dict.yaml") as file:
        second_input_token_to_ids = yaml.load(file, Loader=yaml.FullLoader)

    token_to_ids.update(second_input_token_to_ids)

In [22]:

cfg["train_config"]["device"] = 'cpu'
cfg["mapping_dict"] = mapping_dict
#cfg["results_handler"] = results_handler
model = GeneEmbeddModel(cfg)
#load state dict
model.load_state_dict(torch.load(model_path))
model.push_to_hub(model_name)
model = GeneEmbeddModel.from_pretrained(model_name)

In [23]:
tokenizer = RnaTokenizer(model_max_length=29,model_name=model_name)
tokenizer.add_special_tokens({'pad_token': 'pad'})
x = tokenizer(['AACGAAGCTCGACTTTTAAGG'\
            ,'GTCCACCCCAAAGCGTAGG'])

Tokenizing sequences for yak-hbdx/seq-rev-transforna model...


In [24]:
x

{'input_ids': tensor([[ 8,  4, 16,  3,  8, 10, 11,  1, 12, 16,  3,  4,  1,  9,  9,  9,  6,  8,
         10, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15, 10,  8,  6,  9,  9,  9,
          1,  4,  3, 16, 12,  1, 11, 10,  8,  3, 16,  4,  8,  0,  0,  0,  0,  0,
          0,  0,  0,  0, 21],
        [14, 12,  5, 13,  4,  5,  5,  5, 13,  8,  8, 10, 11, 16, 14,  6, 10, 15,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15, 10,  6, 14, 16, 11, 10,
          8,  8, 13,  5,  5,  5,  4, 13,  5, 12, 14,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0, 19]])}

In [25]:
#save tokenizer and push to hub
tokenizer.push_to_hub(model_name)

CommitInfo(commit_url='https://huggingface.co/Yak-hbdx/Seq-Rev-TransfoRNA/commit/aff4c111f6ab9817ccf1a199f66e9246b92fa2ba', commit_message='Upload tokenizer', commit_description='', oid='aff4c111f6ab9817ccf1a199f66e9246b92fa2ba', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
#load model and tokenizer
model = GeneEmbeddModel.from_pretrained(model_name)
model.eval()
tokenizer = RnaTokenizer.from_pretrained(model_name,model_name=model_name)
output = tokenizer(['AAAGTCGGAGGTTCGAAGACGATCAGATAC','TTTTCGGAACTGAGGCCATGATTAAGAGGG'])
gene_embedd, second_input_embedd, activations,attn_scores_first,attn_scores_second = model(output['input_ids'])
#get the idx of the maximum value in the gene_embedd tensor at each row
class_ids = torch.argmax(activations,dim=1).numpy()
class_labels = model.convert_ids_to_labels(class_ids)
#asset ['18S_bin-38', '18S_bin-33']
assert class_labels == ['18S_bin-38', '18S_bin-33'], print('\033[91m' + 'FAILED')
print('\033[92m' + 'PASSED')

Tokenizing sequences for yak-hbdx/seq-rev-transforna model...
PASSED
